In [1]:
FUNCTION read_dap_MCD43A4,url,sites
; open the DAP file
 dap_file = ncdf_open(url)
; get the latitude and longitude resolutions from the global attributes
 ncdf_attget,dap_file,"geospatial_lat_resolution",lat_res,/global
 ncdf_attget,dap_file,"geospatial_lon_resolution",lon_res,/global
; get the time variable
 time_id = ncdf_varid(dap_file,"time")
 ncdf_varget,dap_file,time_id,time
 nrecs = n_elements(time)
; and the time variable units attribute
 ncdf_attget,dap_file,time_id,"units",value
 time_units = string(value)
; get the year, month and day from the time units string
 result = strsplit(time_units," ",/extract)
 ymd = strsplit(result[2],"-",/extract)
; get the hour, minute and seconds
 hms = strsplit(result[3],":",/extract)
; get the Julian date from the netCDF time
 jul_time = time + JulDay(fix(ymd[1]),fix(ymd[2]),fix(ymd[0]),0,0,0)
; ... and get the year, month, day etc from the Julian date
 CalDat, jul_time, month, day, year, hour, minute, second
;print, year[-1], month[-1], day[-1], hour[-1], minute[-1], second[-1]
; get the latitude and longitude variables from the netCDF file
 lat_id = ncdf_varid(dap_file,"latitude")
 ncdf_varget,dap_file,lat_id,latitude
 lon_id = ncdf_varid(dap_file,"longitude")
 ncdf_varget,dap_file,lon_id,longitude
; NOTE: AusCover DAP files are dimensioned as [longitude,latitude,time]
;                                          eg [19160,14902,365]
 nsites = n_elements(sites.name)
 for i=0,nsites-1 do begin
   print,"Processing site: ",sites.name[i]
; get the latitude and longitude indices
   lat_index = fix(((latitude[0]-sites.latitude[i])/lat_res)+0.5)
   if sites.longitude[i]<0 then sites.longitude[i] = float(360) + sites.longitude[i]
   lon_index = fix(((sites.longitude[i]-longitude[0])/lon_res)+0.5)
; get the offset and count for the data subset
   offset = [lon_index-1,lat_index-1,0]
   count = [3,3,nrecs]
; and now get the BDRF reflectance data
; 459 to 479 nm
   id = ncdf_varid(dap_file,"nbar_0459_0479nm")
   ncdf_varget,dap_file,id,nbar_0459_0479nm,offset=offset,count=count
; 545 to 565 nm
   id = ncdf_varid(dap_file,"nbar_0545_0565nm")
   ncdf_varget,dap_file,id,nbar_0545_0565nm,offset=offset,count=count
; 620 to 670 nm
   id = ncdf_varid(dap_file,"nbar_0620_0670nm")
   ncdf_varget,dap_file,id,nbar_0620_0670nm,offset=offset,count=count
; 841 to 876 nm
   id = ncdf_varid(dap_file,"nbar_0841_0876nm")
   ncdf_varget,dap_file,id,nbar_0841_0876nm,offset=offset,count=count
; 1230 to 1250 nm
   id = ncdf_varid(dap_file,"nbar_1230_1250nm")
   ncdf_varget,dap_file,id,nbar_1230_1250nm,offset=offset,count=count
; 1628 to 1652 nm
   id = ncdf_varid(dap_file,"nbar_1628_1652nm")
   ncdf_varget,dap_file,id,nbar_1628_1652nm,offset=offset,count=count
; 2105 to 2155 nm
   id = ncdf_varid(dap_file,"nbar_2105_2155nm")
   ncdf_varget,dap_file,id,nbar_2105_2155nm,offset=offset,count=count
; quality flag
   id = ncdf_varid(dap_file,"quality")
   ncdf_varget,dap_file,id,quality,offset=offset,count=count
; snow
   id = ncdf_varid(dap_file,"snow")
   ncdf_varget,dap_file,id,snow,offset=offset,count=count
; typical mask
   id = ncdf_varid(dap_file,"typical_mask")
   ncdf_varget,dap_file,id,typical_mask,offset=offset,count=count
; create the data structure
   new = {name:sites.name[i],year:year,month:month,day:day,$
          hour:hour,minute:minute,second:second,$
          nbar_0459_0479nm:nbar_0459_0479nm,$
          nbar_0545_0565nm:nbar_0545_0565nm,$
          nbar_0620_0670nm:nbar_0620_0670nm,$
          nbar_0841_0876nm:nbar_0841_0876nm,$
          nbar_1230_1250nm:nbar_1230_1250nm,$
          nbar_1628_1652nm:nbar_1628_1652nm,$
          nbar_2105_2155nm:nbar_2105_2155nm,$
          quality:quality,snow:snow,$
          typical_mask:typical_mask}
; add this structure to the array
   if i eq 0 then data = replicate(new,nsites) else data[i] = new
 endfor
; close the netCDF file
 ncdf_close,dap_file
; return the data structure
 return,data
END

In [2]:
sites = {name:["Sturt Plains"],latitude:[-17.15090],longitude:[133.35055]}

In [3]:
; define the DAP URL and the file name
dap_path = "http://www.auscover.org.au/thredds/dodsC/auscover/lpdaac-aggregates/c5/v2-nc4/aust/MCD43A4.005/"
file_name = "MCD43A4.aggregated.aust.005.nadir_brdf_adjusted_reflectance.ncml"
; get the full URL and open the netCDF file
url = dap_path+file_name

In [4]:
data = read_dap_MCD43A4(url,sites)

Processing site: Sturt Plains


In [9]:
for i=0,n_elements(data)-1 do begin
 nrecs = n_elements(data[i].year)
; get the decimal year for plotting
 julian_date = JulDay(data[i].month,data[i].day,data[i].year,$
                      data[i].hour,data[i].minute,data[i].second)
 doy = julian_date - JulDay(1,1,data[i].year)
 diy = intarr(nrecs)
 diy = 365
 idx = where((data[i].year mod 4) eq 0,count)
 diy[idx] = 366
 ydy = data[i].year + doy/diy + data[i].hour/(24*diy) + data[i].minute/(24*60*diy)
 window,i
 plot,ydy,data[i].nbar_0459_0479nm[1,1,*],max_value=0.2,min_value=0.0,$
      xrange=[2000,2017],xtickinterval=1,$
      xtitle="Years",ytitle="BDRF Reflectance",$
      title=sites.name[i]
 oplot,ydy,data[i].nbar_0459_0479nm[0,0,*],max_value=0.2,min_value=0.0
 oplot,ydy,data[i].nbar_0459_0479nm[0,1,*],max_value=0.2,min_value=0.0
 oplot,ydy,data[i].nbar_0459_0479nm[0,2,*],max_value=0.2,min_value=0.0
 oplot,ydy,data[i].nbar_0459_0479nm[1,0,*],max_value=0.2,min_value=0.0
 oplot,ydy,data[i].nbar_0459_0479nm[1,2,*],max_value=0.2,min_value=0.0
 oplot,ydy,data[i].nbar_0459_0479nm[2,0,*],max_value=0.2,min_value=0.0
 oplot,ydy,data[i].nbar_0459_0479nm[2,1,*],max_value=0.2,min_value=0.0
 oplot,ydy,data[i].nbar_0459_0479nm[2,2,*],max_value=0.2,min_value=0.0
endfor

In [ ]:
sites = {name:["Calperum","Sturt Plains","Whroo"],$
         latitude:[-34.00206,-17.15090,-36.67305],$
         longitude:[140.58912,133.35055,145.02621]}

In [ ]:
dap_file = ncdf_open(url)
ncdf_attget,dap_file,"geospatial_lat_resolution",lat_res,/global
ncdf_attget,dap_file,"geospatial_lon_resolution",lon_res,/global
time_id = ncdf_varid(dap_file,"time")
ncdf_varget,dap_file,time_id,time
nrecs = n_elements(time)
ncdf_attget,dap_file,time_id,"units",value
time_units = string(value)
result = strsplit(time_units," ",/extract)
ymd = strsplit(result[2],"-",/extract)
hms = strsplit(result[3],":",/extract)
jul_time = time + JulDay(fix(ymd[1]),fix(ymd[2]),fix(ymd[0]),0,0,0)
CalDat, jul_time, month, day, year, hour, minute, second
lat_id = ncdf_varid(dap_file,"latitude")
ncdf_varget,dap_file,lat_id,latitude
lon_id = ncdf_varid(dap_file,"longitude")
ncdf_varget,dap_file,lon_id,longitude
nsites = n_elements(sites.name)

In [ ]:
for i=0,nsites-1 do begin
  print,"Processing site: ",sites.name[i]
  lat_index = fix(((latitude[0]-sites.latitude[i])/lat_res)+0.5)
  if sites.longitude[i]<0 then sites.longitude[i] = float(360) + sites.longitude[i]
  lon_index = fix(((sites.longitude[i]-longitude[0])/lon_res)+0.5)
  offset = [lon_index-1,lat_index-1,0]
  count = [3,3,nrecs]
  id = ncdf_varid(dap_file,"nbar_0459_0479nm")
  ncdf_varget,dap_file,id,nbar_0459_0479nm,offset=offset,count=count